In [12]:
from pytube import YouTube
from tqdm import tqdm
import librosa
import librosa.display
import ffmpeg
import numpy as np
import moviepy
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [5]:
# highlights
yt = YouTube('https://www.youtube.com/watch?v=HF6wnXp3rsE')
# game
# yt = YouTube('https://www.youtube.com/watch?v=GlqyJDKV6eg')
stream = yt.streams.filter().first()

pbar = tqdm(total=stream.filesize)
def progress_fn(self, chunk, *_):
    pbar.update(len(chunk))
yt.register_on_progress_callback(progress_fn)    

stream.download(output_path=".", filename="tmp_highlights") 

 99%|█████████▉| 42590208/42854918 [00:14<00:00, 2931586.96it/s]

'./tmp_highlights.mp4'

100%|██████████| 42854918/42854918 [00:30<00:00, 2931586.96it/s]

In [7]:
ffmpeg.input("tmp.mp4").audio.output("out.wav").run()


(None, None)

In [4]:
data, sampling_rate = librosa.load('out.wav')

In [13]:
print("total shape", data.shape)
print("sampling rate", sampling_rate)

# librosa.display.waveplot(data[0:22050 * 1000], sr=sampling_rate)

data_size = data.shape[0]
data_total_length = ((data_size // sampling_rate) + 1) * sampling_rate
data = np.pad(data, pad_width=(0, data_total_length - data_size), mode='constant')

data_chunks = np.split(data, data_total_length/sampling_rate)

# print(data_chunks)
print(len(data_chunks))

data_sums = np.array(list(map(lambda x : np.sum(np.abs(x)), data_chunks)))

n = 10
top_n_idx = np.argsort(data_sums)[-n:]
top_n_values = [data_sums[i] for i in top_n_idx]

print(top_n_idx)
print(top_n_values)

# Create highlights
highlight_len = 5
top_n_idx = sorted(top_n_idx)
clips = []
for i in range(len(top_n_idx)):
    time = top_n_idx[i]
    clip_name = "test" + str(i) + ".mp4"
    ffmpeg_extract_subclip(
        "tmp.mp4", 
        time - highlight_len, 
        time + highlight_len, 
        targetname=clip_name
    )
    clips.append(VideoFileClip(clip_name))

final_clip = concatenate_videoclips(clips)
final_clip.write_videofile("highlights.mp4")

total shape (87097500,)
sampling rate 22050
3951
[3020 2978 3022 3885 3887 3283 3298 3026 3589 3660]
[1581.0247, 1602.9133, 1609.6527, 1614.9598, 1616.0969, 1657.2307, 1685.9121, 1690.6027, 1744.8236, 1824.0743]
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


chunk:   2%|▏         | 50/2514 [00:00<00:05, 492.35it/s, now=None]

Moviepy - Building video highlights.mp4.
MoviePy - Writing audio in highlightsTEMP_MPY_wvf_snd.mp3


t:   1%|          | 29/3416 [00:00<00:11, 289.83it/s, now=None]      

MoviePy - Done.
Moviepy - Writing video highlights.mp4



Moviepy - Done !
Moviepy - video ready highlights.mp4
